In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os

In [9]:
import pandas as pd

In [10]:
import warnings

# Ignorar avisos de desempenho
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)


In [11]:
os.chdir("/content/gdrive/My Drive/Pesquisa/1 Artigos em desenvolvimento/2024_XX Artigo Residencial Climas.lnk.lnk/01 Desenvolvimento/1 Indicadores de Desempenho Térmico/Outputs_Sim")
filelist = os.listdir()
#filelist

In [12]:
# Intervalos de temperatura para cada cidade
city_data = {
    'BELEM': {'suplimit': 30, 'inflimit': 0},          #INMET 27,16  INTERVALO 3
    'BH': {'suplimit': 26, 'inflimit': 18},             #INMET 21,75  INTERVALO 1
    'BRASILIA': {'suplimit': 26, 'inflimit': 18},       #INMET 21,08  INTERVALO 1
    #'CUIABA': {'suplimit': 28, 'inflimit': 18},         #INMET XX,XX
    'CURITIBA': {'suplimit': 26, 'inflimit': 18},       #INMET 17,38  INTERVALO 1
    'FLORIANOPOLIS': {'suplimit': 26, 'inflimit': 18},  #INMET 20,91  INTERVALO 1
    'FORTALEZA': {'suplimit': 28, 'inflimit': 0},      #INMET 26,65  INTERVALO 2
    #'MACEIO': {'suplimit': 28, 'inflimit': 18},         #INMET XX,XX
    'MANAUS': {'suplimit': 30, 'inflimit': 0},         #INMET 27,28  INTERVALO 3
    #'MARINGA': {'suplimit': 28, 'inflimit': 18},        #INMET XX,XX
    #'NATAL': {'suplimit': 28, 'inflimit': 18},          #INMET XX,XX
    #'POA': {'suplimit': 26, 'inflimit': 18},            #INMET XX,XX
    'RECIFE': {'suplimit': 28, 'inflimit': 0},         #INMET 25,75  INTERVALO 2
    'RJ': {'suplimit': 26, 'inflimit': 18},             #INMET 23,15  INTERVALO 1
    #'SALVADOR': {'suplimit': 28, 'inflimit': 0},       #INMET 25,86  INTERVALO 2
    #'SL': {'suplimit': 28, 'inflimit': 18},             #INMET XX,XX
    'SP': {'suplimit': 26, 'inflimit': 18}             #INMET 19,57  INTERVALO 1
    #'VITORIA': {'suplimit': 26, 'inflimit': 18}         #INMET XX,XX
}


In [13]:
# Listas de UHs e APPs
UH_list = ("UH_01","UH_02","UH_03","UH_04","UH_05","UH_06","UH_07","UH_08","UH_09","UH_10","UH_11","UH_12")
APP_list = ("SALA","DORM1","DORM2")

In [ ]:
# Loop sobre cada arquivo CSV
for file_name in filelist:

    # Extrair o nome da cidade do nome do arquivo
    city = file_name.split('_')[1]
    weatherfile = file_name.split('_')[-1].split('.')[0]
    #print(city)
    #print(weatherfile)

    # Verifica se a cidade está no dicionário de dados da cidade
    if city in city_data:
        df_ac = pd.read_csv(f'AC_{city}_{weatherfile}.csv').copy()
        df_vn = pd.read_csv(f'VN_{city}_{weatherfile}.csv').copy()

        # Removendo espaços em branco no início e no final dos valores na coluna 'Date/Time'
        df_vn['Date/Time'] = df_vn['Date/Time'].str.strip()
        df_ac['Date/Time'] = df_ac['Date/Time'].str.strip()

        # Substituindo "24:00:00" por "00:00:00"
        df_vn['Date/Time'] = df_vn['Date/Time'].str.replace('24:00:00', '00:00:00')
        df_ac['Date/Time'] = df_ac['Date/Time'].str.replace('24:00:00', '00:00:00')

        # Convertendo a coluna 'Date/Time' para o formato datetime
        df_vn['Date/Time'] = pd.to_datetime(df_vn['Date/Time'], format='%m/%d  %H:%M:%S')
        df_ac['Date/Time'] = pd.to_datetime(df_ac['Date/Time'], format='%m/%d  %H:%M:%S')

        # Ajustando a data para o próximo dia quando o horário é "00:00:00"
        next_day_mask = df_vn['Date/Time'].dt.hour == 0
        df_vn.loc[next_day_mask, 'Date/Time'] += pd.Timedelta(days=1)
        next_day_mask = df_ac['Date/Time'].dt.hour == 0
        df_ac.loc[next_day_mask, 'Date/Time'] += pd.Timedelta(days=1)

        # Intervalo de temperatura superior e inferior para a cidade atual
        suplimit = city_data[city]['suplimit']
        inflimit = city_data[city]['inflimit']

        results_monthly = []
        results_daily = []
        results_yearly = []

        for UH in UH_list:
            for APP in APP_list:
                # Ajustando o nome da coluna do schedule value
                if APP.startswith("DORM"):
                    schedule_value_col = f'SCH_OCUP_DORM:Schedule Value [](Hourly) '
                else:
                    schedule_value_col = f'SCH_OCUP_{APP}:Schedule Value [](Hourly)'

                # Filtrar dados para a combinação atual de UH e APP
                filtered_data_cooling = df_ac[(df_vn[f'{UH}_{APP}:Zone Operative Temperature [C](Hourly)'] >= suplimit) & (df_vn[schedule_value_col] != 0)].copy()
                filtered_data_heating = df_ac[(df_vn[f'{UH}_{APP}:Zone Operative Temperature [C](Hourly)'] <= inflimit) & (df_vn[schedule_value_col] != 0)].copy()
                occupied_data = df_vn[df_vn[schedule_value_col] != 0].copy()

                # Calcular resolução mensal
                space_cooling_monthly = (filtered_data_cooling.groupby(pd.Grouper(key='Date/Time', freq='M'))
                                          [f'{UH}_{APP} IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Cooling Energy [J](Hourly)']
                                          .sum() / 3600000).reset_index()
                space_heating_monthly = (filtered_data_heating.groupby(pd.Grouper(key='Date/Time', freq='M'))
                                          [f'{UH}_{APP} IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Heating Energy [J](Hourly)']
                                          .sum() / 3600000).reset_index()
                space_op_temp_monthly = occupied_data.groupby(pd.Grouper(key='Date/Time', freq='M'))[f'{UH}_{APP}:Zone Operative Temperature [C](Hourly)'].max().reset_index()

                # Juntar os resultados em uma única tabela
                space_result_month = pd.concat([space_cooling_monthly.set_index('Date/Time'),
                                            space_heating_monthly.set_index('Date/Time'),
                                            space_op_temp_monthly.set_index('Date/Time')], axis=1)

                # Renomear as colunas
                space_result_month.rename(columns={f'{UH}_{APP} IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Cooling Energy [J](Hourly)': f'{UH}_{APP} Cooling Energy (kWh)',
                                              f'{UH}_{APP} IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Heating Energy [J](Hourly)': f'{UH}_{APP} Heating Energy (kWh)',
                                              f'{UH}_{APP}:Zone Operative Temperature [C](Hourly)': f'{UH}_{APP} Max Operative Temperature (C)'}, inplace=True)

                # Adicionar os resultados desta combinação de UH e APP à lista
                results_monthly.append(space_result_month)

                ## Resolução diária

                # Calcular resolução diária
                space_cooling_daily = (filtered_data_cooling.groupby(pd.Grouper(key='Date/Time', freq='D'))
                                        [f'{UH}_{APP} IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Cooling Energy [J](Hourly)']
                                        .sum() / 3600000).reset_index()
                space_heating_daily = (filtered_data_heating.groupby(pd.Grouper(key='Date/Time', freq='D'))
                                        [f'{UH}_{APP} IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Heating Energy [J](Hourly)']
                                        .sum() / 3600000).reset_index()
                space_op_temp_daily = occupied_data.groupby(pd.Grouper(key='Date/Time', freq='D'))[f'{UH}_{APP}:Zone Operative Temperature [C](Hourly)'].max().reset_index()

                # Juntando os resultados em uma única tabela
                space_result_day = pd.concat([space_cooling_daily.set_index('Date/Time'),
                                            space_heating_daily.set_index('Date/Time'),
                                            space_op_temp_daily.set_index('Date/Time')], axis=1)

                # Renomeando as colunas
                space_result_day.rename(columns={f'{UH}_{APP} IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Cooling Energy [J](Hourly)': f'{UH}_{APP} Cooling Energy (kWh)',
                                              f'{UH}_{APP} IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Heating Energy [J](Hourly)': f'{UH}_{APP} Heating Energy (kWh)',
                                              f'{UH}_{APP}:Zone Operative Temperature [C](Hourly)': f'{UH}_{APP} Max Operative Temperature (C)'}, inplace=True)

                # Adicionar os resultados desta combinação de UH e APP à lista
                results_daily.append(space_result_day)

                ## Resolução anual

                # Calcular resolução anual
                space_cooling_yearly = (filtered_data_cooling.groupby(pd.Grouper(key='Date/Time', freq='Y'))
                                        [f'{UH}_{APP} IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Cooling Energy [J](Hourly)']
                                        .sum() / 3600000).reset_index()
                space_heating_yearly = (filtered_data_heating.groupby(pd.Grouper(key='Date/Time', freq='Y'))
                                        [f'{UH}_{APP} IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Heating Energy [J](Hourly)']
                                        .sum() / 3600000).reset_index()
                space_op_temp_yearly = occupied_data.groupby(pd.Grouper(key='Date/Time', freq='Y'))[f'{UH}_{APP}:Zone Operative Temperature [C](Hourly)'].max().reset_index()

                # Juntando os resultados em uma única tabela
                space_result_yearly = pd.concat([space_cooling_yearly.set_index('Date/Time'),
                                            space_heating_yearly.set_index('Date/Time'),
                                            space_op_temp_yearly.set_index('Date/Time')], axis=1)

                # Renomeando as colunas
                space_result_yearly.rename(columns={f'{UH}_{APP} IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Cooling Energy [J](Hourly)': f'{UH}_{APP} Cooling Energy (kWh)',
                                              f'{UH}_{APP} IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Heating Energy [J](Hourly)': f'{UH}_{APP} Heating Energy (kWh)',
                                              f'{UH}_{APP}:Zone Operative Temperature [C](Hourly)': f'{UH}_{APP} Max Operative Temperature (C)'}, inplace=True)

                # Adicionar os resultados desta combinação de UH e APP à lista
                results_yearly.append(space_result_yearly)

        # Juntar todas as tabelas de resultados em uma única tabela final
        final_result_day = pd.concat(results_daily, axis=1)
        final_result_month = pd.concat(results_monthly, axis=1)
        final_result_year = pd.concat(results_yearly, axis=1)

        # Excluir a última linha do DataFrame final
        final_result_day = final_result_day.iloc[:-1].copy()
        final_result_month = final_result_month.iloc[:-1].copy()
        final_result_year = final_result_year.iloc[:-1].copy()

        # Calcular a soma dos valores de resfriamento e aquecimento e temperatura operativa máxima para cada UH em cada linha
        for UH in UH_list:
            cooling_cols = [f'{UH}_{APP} Cooling Energy (kWh)' for APP in APP_list]
            heating_cols = [f'{UH}_{APP} Heating Energy (kWh)' for APP in APP_list]
            operative_temp_cols = [f'{UH}_{APP} Max Operative Temperature (C)' for APP in APP_list]

            final_result_day.loc[:, f'{UH} Total Cooling Energy (kWh)'] = final_result_day[cooling_cols].sum(axis=1)
            final_result_day.loc[:, f'{UH} Total Heating Energy (kWh)'] = final_result_day[heating_cols].sum(axis=1)
            final_result_day.loc[:, f'{UH} Max Operative Temperature (C)'] = final_result_day[operative_temp_cols].max(axis=1)

            final_result_month.loc[:, f'{UH} Total Cooling Energy (kWh)'] = final_result_month[cooling_cols].sum(axis=1)
            final_result_month.loc[:, f'{UH} Total Heating Energy (kWh)'] = final_result_month[heating_cols].sum(axis=1)
            final_result_month.loc[:, f'{UH} Max Operative Temperature (C)'] = final_result_month[operative_temp_cols].max(axis=1)

            final_result_year.loc[:, f'{UH} Total Cooling Energy (kWh)'] = final_result_year[cooling_cols].sum(axis=1)
            final_result_year.loc[:, f'{UH} Total Heating Energy (kWh)'] = final_result_year[heating_cols].sum(axis=1)
            final_result_year.loc[:, f'{UH} Max Operative Temperature (C)'] = final_result_year[operative_temp_cols].max(axis=1)

            # Calcular Total Loads somando Heating e Cooling
            final_result_day[f'{UH} Total Loads (kWh)'] = final_result_day[f'{UH} Total Heating Energy (kWh)'] + final_result_day[f'{UH} Total Cooling Energy (kWh)']
            final_result_month[f'{UH} Total Loads (kWh)'] = final_result_month[f'{UH} Total Heating Energy (kWh)'] + final_result_month[f'{UH} Total Cooling Energy (kWh)']
            final_result_year[f'{UH} Total Loads (kWh)'] = final_result_year[f'{UH} Total Heating Energy (kWh)'] + final_result_year[f'{UH} Total Cooling Energy (kWh)']

        final_result_day.to_csv(f'../NBR15575_Indicadores_DifResolucoes/results/{city}_{weatherfile}_result_day.csv')
        final_result_month.to_csv(f'../NBR15575_Indicadores_DifResolucoes/results/{city}_{weatherfile}_result_month.csv')
        final_result_year.to_csv(f'../NBR15575_Indicadores_DifResolucoes/results/{city}_{weatherfile}_result_year.csv')

        print(f'{city}_{weatherfile}')
    else:
      print(f'O intervalo de temperatura para a cidade {city} não está definido.')